### WESAD, CASE

In [7]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr_c
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE,
# ]
] + train.Metrics.STATISTICAL

model_phases_case = dr_c.CLIPS

model_phases_wesad = dr_w.Phases.PHASE_ORDER

case_label_type = dr_c.SelfReports.AROUSAL
wesad_label_type = "stai"

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

threshold = "fixed"
test_size = 1.0

percent_of_target_dataset = 0.0

temp_a, _ = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_case_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_case_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

temp_a, _ = train.Train_WESAD.get_wesad_data([train.Metrics.BPM], phases=dr_w.Phases.PHASE_ORDER, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_wesad_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_wesad_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [9]:
# TRAIN ON WESAD AND TEST ON CASE -- ALL
importlib.reload(train)
importlib.reload(dr_c)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(59)

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_wesad_case = {}
ensemble_weights_wesad = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_wesad_case[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_a, y_a, n_features=5)
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_wesad_case, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights_wesad[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.92826428 0.92482034 0.9402201  0.93831127 0.93196611 0.91955091
 0.91458427 0.93684655 0.93716824 0.93528416 0.92714009 0.91952364
 0.94226753 0.93737082 0.93833986 0.92548127 0.9234897  0.93801062
 0.93398874 0.9296093  0.94366151 0.92989882 0.94158314 0.93547972
 0.92939816        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 5, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [348 126], Predictions: [0 1], [ 68 406]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [348 126], Predictions: [0 1], [340 134]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'ecg_rms', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'eda_rms']
Model RF, Actual: [0 1], [348 126], Predictions: [0 1], [323 151]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'ecg_rms' 

In [10]:
# TRAIN ON CASE AND TEST ON WESAD -- ALL
importlib.reload(train)
importlib.reload(dr_c)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(59)

x_a, y_a = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_case_wesad = {}
ensemble_weights_case = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_case_wesad[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_a, y_a, n_features=5)
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_case_wesad, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights_case[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.81108997 0.81696817 0.81434554 0.81540076 0.81974026 0.80808611
 0.81773226 0.81639795 0.81601219 0.81390153 0.81229907 0.82388996
 0.82246858 0.82131752 0.81886405 0.81662837 0.82457    0.82398466
 0.81980417 0.82164626 0.79986079 0.81276637 0.82472104 0.82428514
 0.82639251        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 5, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [428  95], Predictions: [0 1], [  2 521]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [428  95], Predictions: [0 1], [ 78 445]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'ecg_rms', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'eda_rms']
Model RF, Actual: [0 1], [428  95], Predictions: [0 1], [ 41 482]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'ec

In [11]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr_c)
importlib.reload(dr_w)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier

random.seed(26)

percent_of_target_dataset = 0.0
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds

In [14]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr_c)
importlib.reload(dr_w)
importlib.reload(dt)

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

random.seed(26)

voting_type = "weighted_avg"
# voting_type = "majority_vote"

test_size = 0.0
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

subjects = list(np.unique(x_b.loc[:, "subject"]))
train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
x_train = pd.concat([x_a, x_train_addition])
y_train = pd.concat([y_a, y_train_addition])
x_test = x_b[~x_b["subject"].isin(train_subjects)]
y_test = y_b[~y_b["subject"].isin(train_subjects)]

y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models_wesad_case[name] for name in ensemble_models}
weights = [ensemble_weights_wesad[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
# weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)

print("Train WESAD, test CASE")
print(f"WESAD size: {x_train.shape[0]}\n{y_train.value_counts()}\nCASE size: {x_test.shape[0]}\n{y_test.value_counts()}")
print("Predictions:")
print(f"{np.array(np.unique(y_pred, return_counts=True)).T}")
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

###############################################################################################################

x_a, y_a = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

subjects = list(np.unique(x_b.loc[:, "subject"]))
train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
x_train = pd.concat([x_a, x_train_addition])
y_train = pd.concat([y_a, y_train_addition])
x_test = x_b[~x_b["subject"].isin(train_subjects)]
y_test = y_b[~y_b["subject"].isin(train_subjects)]

y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models_case_wesad[name] for name in ensemble_models}
weights = [ensemble_weights_case[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
# weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)

print("Train CASE, test WESAD")
print(f"CASE size: {x_train.shape[0]}\n{y_train.value_counts()}\nWESAD size: {x_test.shape[0]}\n{y_test.value_counts()}")
print("Predictions:")
print(f"{np.array(np.unique(y_pred, return_counts=True)).T}")
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

Train WESAD, test CASE
WESAD size: 523
0    428
1     95
Name: label, dtype: int64
CASE size: 474
0    348
1    126
Name: label, dtype: int64
Predictions:
[[  0 319]
 [  1 155]]
	Accuracy: 0.6392405063291139
	Precision: 0.3548387096774194
	Recall: 0.4365079365079365
	F1-score: 0.39145907473309605
	AUC score: 0.574575807334428
----------------------------------------
Train CASE, test WESAD
CASE size: 474
0    348
1    126
Name: label, dtype: int64
WESAD size: 523
0    428
1     95
Name: label, dtype: int64
Predictions:
[[  0 158]
 [  1 365]]
	Accuracy: 0.4416826003824092
	Precision: 0.23013698630136986
	Recall: 0.8842105263157894
	F1-score: 0.36521739130434777
	AUC score: 0.6138342351205115
----------------------------------------
